In [ ]:
#| default_exp datasets.kadik10k

# KADID10K

> Building a `tf.data.Dataset` for Kadik10k.

In [ ]:
#| hide
import os; os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [ ]:
#| export
from pathlib import Path
from typing import List

import pandas as pd
import tensorflow as tf
import cv2

After setting up the path to the directory and loading the corresponding `.csv` file, we need to create a generator that will iterate over the dataframe, load and return a 3-tuple: `(Reference Image, Distorted Image, DMOS)`. When can the pass that generator into a `tf.data.Dataset.from_generator()` to build the `Dataset` object:

In [ ]:
#| export
class KADIK10K():
    """Builder for the KADIK10K dataset"""

    def __init__(self,
                 path, # Path to the root directory of the dataset.
                 exclude_imgs: List[int] = None, # Image ID's to exclude.
                 exclude_dist: List[int] = None, # Distortion ID's to exclude.
                 exclude_ints: List[int] = None, # Distortion Intensities ID's to exclude.
                 num_parallel_calls: int = tf.data.AUTOTUNE, # Number of parallel calls when loading the images.
                 ):
        self.path_root = Path(path) if isinstance(path, str) else path
        self.path_csv = self.path_root/"dmos.csv"
        self.path_images = self.path_root/"images"
        self.data = self.load_data(self.path_csv, exclude_imgs, exclude_dist, exclude_ints)
        self.paths_ref = [str(self.path_images/p) for p in self.data["ref_img"]]
        self.paths_dist = [str(self.path_images/p) for p in self.data["dist_img"]]
        self.num_parallel_calls = num_parallel_calls

    @property
    def dataset(self):
        """tf.data.Dataset object built from the TID2013 dataset."""
        return tf.data.Dataset.from_tensor_slices((self.paths_ref, self.paths_dist, self.data["dmos"]))\
                              .map(self.preprocess, num_parallel_calls=self.num_parallel_calls)

    @staticmethod
    def preprocess(path_ref,
                   path_dist,
                   mos,
                   ):
        img_ref = tf.io.read_file(path_ref)
        img_dist = tf.io.read_file(path_dist)

        img_ref = tf.image.decode_png(img_ref, channels=3)
        img_dist = tf.image.decode_png(img_dist, channels=3)

        img_ref = tf.image.convert_image_dtype(img_ref, dtype=tf.float32)
        img_dist = tf.image.convert_image_dtype(img_dist, dtype=tf.float32)

        return img_ref, img_dist, mos

    def load_data(self,
                  path,
                  exclude_imgs,
                  exclude_dist,
                  exclude_ints,
                  ):
        data = pd.read_csv(self.path_csv)
        data = data[~data.Reference_ID.isin(exclude_imgs)] if exclude_imgs is not None else data
        data = data[~data.Reference_ID.isin(exclude_dist)] if exclude_dist is not None else data
        data = data[~data.Reference_ID.isin(exclude_ints)] if exclude_ints is not None else data
        return data


In [ ]:
# l = KADIK10K(path = Path("/media/disk/databases/BBDD_video_image/Image_Quality/KADIK10K"))
l = KADIK10K(path = Path("/lustre/ific.uv.es/ml/uv075/Databases/IQA/KADIK10K/"))

In [ ]:
l.data.head()

,dist_img,ref_img,dmos,var
0,I01_01_01.png,I01.png,4.57,0.496
1,I01_01_02.png,I01.png,4.33,0.869
2,I01_01_03.png,I01.png,2.67,0.789
3,I01_01_04.png,I01.png,1.67,0.596
4,I01_01_05.png,I01.png,1.10,0.300


In [ ]:
for a, b, c in l.dataset:
    break
assert a.shape == b.shape

2023-09-13 15:19:51.686217: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-09-13 15:19:51.686343: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: mlui02.ific.uv.es
2023-09-13 15:19:51.686379: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: mlui02.ific.uv.es
2023-09-13 15:19:51.686679: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 535.54.3
2023-09-13 15:19:51.686785: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 535.54.3
2023-09-13 15:19:51.686809: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 535.54.3
2023-09-13 15:19:51.688098: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CP

And we can benchmark it to finish:

In [ ]:
#| eval: false
from tqdm.auto import tqdm
for a, b, c in tqdm(l.dataset): pass

  0%|          | 0/10125 [00:00<?, ?it/s]

## Extending the base `.csv`

> We can extend the given `.csv` to separate information regarding the distortion and it's intensity to add more flexibility to the posterior analysis.

In [ ]:
#| hide
import re

In [ ]:
def extract_distortion_intensity(row):
    """Extracts the distortion and the intensity from the distorted image filename."""
    distortion, intensity = re.findall(r"I\d+_(\d+)_(\d+).png", row.dist_img)[0]
    return {"distortion": distortion, "intensity": intensity}

In [ ]:
l.data = pd.concat([l.data, l.data.apply(extract_distortion_intensity, axis=1, result_type="expand")], axis=1)
l.data.head()

,dist_img,ref_img,dmos,var,distortion,intensity
0,I01_01_01.png,I01.png,4.57,0.496,01,01
1,I01_01_02.png,I01.png,4.33,0.869,01,02
2,I01_01_03.png,I01.png,2.67,0.789,01,03
3,I01_01_04.png,I01.png,1.67,0.596,01,04
4,I01_01_05.png,I01.png,1.10,0.300,01,05


We'll save it as a different file to avoid breaking currently working code:

In [ ]:
l.data.to_csv("/media/disk/databases/BBDD_video_image/Image_Quality/KADIK10K/dmos_extended.csv")